In [256]:
import pandas as pd
import numpy as np

In [260]:
!pip install pickle-mixin

  Preparing metadata (setup.py) ... done
  DEPRECATION: pickle-mixin is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for pickle-mixin ... done

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [194]:
df = pd.read_csv('price.csv')
df.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [195]:
df.shape

(892, 6)

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [197]:
df['year'].unique()

array(['2007', '2006', '2018', '2014', '2015', '2012', '2013', '2016',
       '2010', '2017', '2008', '2011', '2019', '2009', '2005', '2000',
       '...', '150k', 'TOUR', '2003', 'r 15', '2004', 'Zest', '/-Rs',
       'sale', '1995', 'ara)', '2002', 'SELL', '2001', 'tion', 'odel',
       '2 bs', 'arry', 'Eon', 'o...', 'ture', 'emi', 'car', 'able', 'no.',
       'd...', 'SALE', 'digo', 'sell', 'd Ex', 'n...', 'e...', 'D...',
       ', Ac', 'go .', 'k...', 'o c4', 'zire', 'cent', 'Sumo', 'cab',
       't xe', 'EV2', 'r...', 'zest'], dtype=object)

In [198]:
# df['Price'].unique()

## Quality 
- year has many non-year values 
- year should be in integer, not in object 
- price has 'ask for price' row 
- convert price to integer
- kms_driven has nan values 
- kms_driven object to integer 
- fuel has nan values
- keep first 3 words of name 

## Data Cleaning

In [199]:
backup_df = df.copy()

In [200]:
# string operation to see if year has numeric value 
# here all the (LITERAL) string values will be removed from our column
df = df[df['year'].str.isnumeric()]

In [201]:
df['year']=df['year'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 842 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        842 non-null    object
 1   company     842 non-null    object
 2   year        842 non-null    int64 
 3   Price       842 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: int64(1), object(5)
memory usage: 46.0+ KB


In [202]:
# Exclude 'Ask for price' from the price col

df = df[df['Price'] != 'Ask For Price']

In [203]:
df['Price'] = df['Price'].str.replace(',', '').astype(int)

In [204]:
df['kms_driven'] = df['kms_driven'].str.split(' ').str.get(0).str.replace(',', '') 

In [205]:
df = df[df['kms_driven'].str.isnumeric()]

In [206]:
df['kms_driven'] = df['kms_driven'].astype(int)

In [207]:
# exclude the row that contains null value in fuel type
# ~ will exclude the row
df = df[~df['fuel_type'].isna()]

In [208]:
# keep first three values of name
df['name'] = df['name'].str.split(' ').str.slice(0, 3).str.join(' ')

In [209]:
df = df.reset_index(drop = True)

In [210]:
df.describe()

,year,Price,kms_driven
count,816.000000,8.160000e+02,816.000000
mean,2012.444853,4.117176e+05,46275.531863
std,4.002992,4.751844e+05,34297.428044
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,27000.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.912500e+05,56818.500000
max,2019.000000,8.500003e+06,400000.000000


We can see the we have 75% of the cars to have the value of less than 5lakhs. So the maximum value is an outlier as it is 85lakhs

In [211]:
# get many cars above 60 lakhs
# df[df['Price'] >6e6]

In [212]:
# remove the outlier 
df = df[df['Price']<6e6].reset_index(drop= True)

In [213]:
# store this data into a csv file
df.to_csv('Cleaned_csv_file')

## Model Building

In [223]:
# get the features and the targets

X = df.drop(columns = 'Price')
y = df['Price']

In [265]:
# split the data 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((652, 5), (652,), (163, 5), (163,))

In [266]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [267]:
ohe = OneHotEncoder()

''' This step essentially prepares the OneHotEncoder to transform 
    the categorical data into numerical values that can be used in machine learning models. 
'''

ohe.fit(X[['name', 'company', 'fuel_type']])

OneHotEncoder()

In [268]:
''' - make_column_transformer is widely used when we have more than one classifier to use to transform
    - it contains a one hot encoder to deal with three columns. and the remainder is "passthrough", 
    - so it will concatenate the rest columns to the end of the one hot encoder output.
'''

col_tra = make_column_transformer((OneHotEncoder(categories = ohe.categories_), ['name', 'company','fuel_type']), 
                                  remainder = 'passthrough')

In [269]:
lr = LinearRegression()

In [270]:
pipe = make_pipeline(col_tra, lr)

In [271]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [272]:
y_pred = pipe.predict(X_test)

In [273]:
r2_score(y_test, y_pred)

0.6589847820860448

In [254]:
# We can also check in which random_state the r2_score is greater
scores = [] # to keep the r2_scores of each random_state
for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = i)
    ohe = OneHotEncoder()
    ohe.fit(X[['name', 'company', 'fuel_type']])
    lr = LinearRegression()
    pipe = make_pipeline(col_tra, lr)
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    scores.append(r2_score(y_test, y_pred))

In [257]:
np.argmax(scores)

433

In [258]:
scores[np.argmax(scores)]

0.8456268648196694

Using the random_state that gives us maximum r2_score

In [259]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = np.argmax(scores))
ohe = OneHotEncoder()
ohe.fit(X[['name', 'company', 'fuel_type']])
lr = LinearRegression()
pipe = make_pipeline(col_tra, lr)
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
r2_score(y_test, y_pred)

0.8456268648196694

DUMP THE PIPELINE

Dumping a machine learning pipeline means persisting it to disk, usually as a binary file or a human-readable text file, so that it can be loaded and used later.

In [262]:
import pickle

In [263]:
pickle.dump(pipe, open ('LinearRegressionModel.pkl', 'wb'))

In [264]:
pipe.predict(pd.DataFrame([['Hyundai i20', 'Hyundai', 2018, 100, 'Petrol']], columns=['name', 'company','year','kms_driven', 'fuel_type']))

array([488316.09902509])